In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import xgboost as xgb
from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import MinMaxScaler

data = pd.read_csv("/Users/uppalavenkatesh/Documents/StockMarketData.csv") 
scaler = MinMaxScaler(feature_range=(0, 1))
data["StockValue"] = scaler.fit_transform(data["StockValue"].values.reshape(-1, 1))
X_lstm = data["StockValue"].values.reshape(-1, 1)
y_lstm = data["StockValue"].shift(-1).dropna()  
X_lstm = X_lstm[:-1]  

X_lstm_train, X_lstm_test, y_lstm_train, y_lstm_test = train_test_split(X_lstm, y_lstm, test_size=0.2, shuffle=False)


model_lstm = Sequential([
    tf.keras.Input(shape=(X_lstm_train.shape[1], 1)),  
    LSTM(50, activation='relu'),
    Dense(1)
])
model_lstm.compile(optimizer='adam', loss='mse')
model_lstm.fit(X_lstm_train, y_lstm_train, epochs=10, verbose=1)
lstm_preds = model_lstm.predict(X_lstm_test)
arima_model = ARIMA(data["StockValue"], order=(5, 1, 0))  
arima_model_fit = arima_model.fit()
arima_forecast = arima_model_fit.forecast(steps=len(y_lstm_test))

X_xgb = data["StockValue"].values.reshape(-1, 1)  
y_xgb = data["StockValue"].shift(-1).dropna()  
X_xgb = X_xgb[:-1]  

X_xgb_train, X_xgb_test, y_xgb_train, y_xgb_test = train_test_split(X_xgb, y_xgb, test_size=0.2, shuffle=False)
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100, max_depth=3)
xgb_model.fit(X_xgb_train, y_xgb_train)
xgb_preds = xgb_model.predict(X_xgb_test)

lstm_preds = lstm_preds[-50:].flatten()  
arima_forecast = np.array(arima_forecast[-50:]).flatten() 
xgb_preds = xgb_preds[-50:].flatten()  
dates = data["Date"][-50:].reset_index(drop=True)
actual_values = data["StockValue"][-50:].reset_index(drop=True)


pred_df = pd.DataFrame({
    "Date": dates,  
    "Actual_StockValue": actual_values,
    "LSTM_Prediction": lstm_preds,
    "ARIMA_Prediction": arima_forecast,
    "XGBoost_Prediction": xgb_preds
})
pred_df.to_csv("StockModelPredictions.csv", index=False)
print("Predictions saved successfully!")


Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 492us/step - loss: 0.1868
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 444us/step - loss: 0.0540
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 446us/step - loss: 0.0258
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 547us/step - loss: 0.0109
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 468us/step - loss: 0.0027
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 459us/step - loss: 0.0010
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 446us/step - loss: 6.2687e-04
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 447us/step - loss: 7.7517e-04
Epoch 9/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 441us/step - loss: 6.8972e-04
Epoch 10/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 441us/step - loss: 8.2401e-04
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
